In [ ]:
!ls /eos/cms/store/group/dpg_hgcal/tb_hgcal/2024/hgcalrd/Test/calibrations/
!ls /eos/cms/store/group/dpg_hgcal/tb_hgcal/2024/hgcalrd/Test/calibrations_closure

In [ ]:
def getCalibDirectories(run : int) -> list:
    basedir='/eos/cms/store/group/dpg_hgcal/tb_hgcal/2024/hgcalrd/Test'
    return [f'{basedir}/calibrations/Run{run}', f'{basedir}/calibrations_closure/Run{run}']
 
runs=[1726225188, 1726339135, 1726346360, 1726512750, 1726304466, 1726340355, 1726428151]
outdir='/eos/user/p/psilva/www/HGCal/TB2024/Pedestals'
import os
os.system(f'mkdir -p {outdir}')

In [ ]:
import ROOT
def showValidChannels(run : int, outdir : str):

    calibdir = getCalibDirectories(run)[0]
    url=f'{calibdir}/pedestals_hexplots.root'

    histos={}
    fIn=ROOT.TFile.Open(url,'READ')
    for k in fIn.GetListOfKeys():
        mod=k.GetName()
        i = len(histos)
        histos[mod]=fIn.Get(f'{mod}/Valid').Clone(f'hexplot_{i}')
        histos[mod].SetDirectory(0)
        histos[mod].GetXaxis().SetLabelOffset(999)
        histos[mod].GetXaxis().SetLabelSize(0)
        histos[mod].GetYaxis().SetLabelOffset(999)
        histos[mod].GetYaxis().SetLabelSize(0)
        histos[mod].GetZaxis().SetRangeUser(0,1)
    fIn.Close()

    ROOT.gStyle.SetPalette(ROOT.kDarkBodyRadiator)
    ROOT.gStyle.SetOptStat(0)
    ROOT.gStyle.SetOptTitle(0)
    c=ROOT.TCanvas('c','c',400,600)
    c.Divide(2,3)
    txt=ROOT.TLatex()
    txt.SetTextAlign(22)
    txt.SetTextFont(42)
    txt.SetTextSize(0.1)
    i=0
    for k in histos:
        p = c.cd(i+1)
        p.SetTopMargin(0)
        p.SetLeftMargin(0)
        p.SetRightMargin(0)
        p.SetBottomMargin(0)
        histos[k].Draw('colz')
        txt.DrawLatexNDC(0.5,0.15,histos[k].GetTitle())
        i+=1
    c.cd()
    txt.SetTextSize(0.06)
    txt.DrawLatexNDC(0.5,0.95,f'Run {run}')
    c.Modified()
    c.Update() 
    c.SaveAs(f'{outdir}/Valid_Run{run}.png')
    
for r in runs:
    showValidChannels(r,outdir)

In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)
from scipy.optimize import curve_fit 

def getPedestalDataset(run : int) :

    #cell area information
    with open('cellareas.json') as fin:
        cellareas = json.load(fin)
    def _assignCellArea(row):    
        typecodeNoBP=row['Typecode'][0:4].replace('-','_')
        return cellareas[typecodeNoBP]['SF']
    
    #read pedestal json
    calibdir, closuredir = getCalibDirectories(run)
    with open(f'{calibdir}/pedestals.json') as fin:
        data = pd.read_json(fin,orient='index')
        cols = data.columns
        data.reset_index(inplace=True)
        data = data.rename({'index':'Typecode'},axis=1)
        data['AreaSF'] = data.apply(_assignCellArea, axis=1)
        data['Typecode'] = data['Typecode'].str.replace('-','_')
        data = data.explode(list(cols)+['AreaSF'])
        data['ROC'] = data['Channel']/74
        data = data.astype({'ROC':int})
    
    closureurl=f'{closuredir}/pedestalsclosure.json'
    closuredata = None
    if os.path.isfile(closureurl):
        with open(closureurl) as fin:
            closuredata = pd.read_json(fin,orient='index')
            cols = closuredata.columns
            closuredata.reset_index(inplace=True)
            closuredata = closuredata.rename({'index':'Typecode'},axis=1)
            closuredata['Typecode'] = closuredata['Typecode'].str.replace('-','_')
            def _addChannel(row):
                nch=len(row['En_rms'])
                return [i for i in range(nch)]
            closuredata['Channel'] = closuredata.apply(_addChannel,axis=1)
            closuredata = closuredata.explode(list(cols)+['Channel'])            
        data = data.merge(closuredata, on=['Typecode','Channel'], how='inner')
    
    return data

data = dict( [(run,getPedestalDataset(run)) for run in runs])

In [ ]:
for r, df in data.items():
    mask=(df['Valid']==0)
    print(f'Percentage of invalid channels in run {r}: {100*mask.sum()/df.shape[0]}')
    if mask.sum():
        print(df[mask][['Typecode','Channel']])

In [ ]:
plt.style.use(hep.style.CMS)

def showPlotComparisonsFor(df,var,varLabel,bins,outfile):
    
    if not var in df.columns: return
    
    fig, ax = plt.subplots(1,3,figsize=(18,6))
    
    kwargs={'histtype':'fill','edgecolor':'black','alpha':0.5,'stack':False,'lw':2}
    
    #plot by cell area
    histos=[]
    labels=[]
    for alims in [(0,0.6),(0.6,0.9),(0.9,1.5)]:                
        mask = (df['Valid']==1) & (df['AreaSF']>alims[0]) & (df['AreaSF']<alims[1])
        m=df[mask][var].mean()
        histos.append( np.histogram(df[mask][var],bins=bins)[0] )
        labels.append(rf'${alims[0]:3.1f}<A/A_{{full}}<{alims[1]:3.1f}$ : {m:3.2f}')        
    hep.histplot(histos,bins=bins,**kwargs,ax=ax[0],label=labels)
    
    #plot by module type
    mask = (df['Valid']==True) & (df['AreaSF']>0)
    histos=[]
    labels=[]
    for mod, group in df[mask].groupby('Typecode'):
        m=group[var].mean()
        histos.append( np.histogram(group[var],bins=bins)[0] )
        labels.append(f'{mod} : {m:3.2f}')
    hep.histplot(histos,bins=bins,**kwargs,ax=ax[1],label=labels)
            
    #plot by ROC
    histos=[]
    labels=[]
    for roc, group in df[mask].groupby('ROC'):
        m=group[var].mean()
        histos.append( np.histogram(group[var],bins=bins)[0] )
        labels.append(f'ROC {roc}: {m:3.2f}')
    hep.histplot(histos,bins=bins,**kwargs,ax=ax[2],label=labels)
                  
    for i in range(3):
        ax[i].legend(fontsize=12,loc='upper left')
        ax[i].grid()
        ax[i].set_xlabel(varLabel)
    fig.tight_layout
    plt.savefig(f'{outfile}_{var}.png')
    plt.close()

for r, df in data.items():
    for var, varLabel, bins in [
        ('cm2_slope',  r'ADC vs $CM_{2}$ slope',   np.linspace(-0.2,1.0,50)),
        ('cm4_slope',  r'ADC vs $CM_{4}$ slope',   np.linspace(-0.2,1.0,50)),
        ('cmall_slope',r'ADC vs $CM_{all}$ slope', np.linspace(-0.2,1.0,50)),
        ('adcm1_slope',r'ADC vs ADC(BX-1) slope', np.linspace(-0.2,0.2,50)),                            
        ('ADC_rms','RMS(ADC)',np.linspace(0,5,50)),
        ('En_ped','Residual pedestal',np.linspace(-5,5,50)),
        ('En_rms','RMS(RecHit)',np.linspace(0,5,50)),
        ('CM_slope','Residual $CM_{2}$ slope',np.linspace(-0.2,1,50)),
        ('CNF','Coeherent noise fraction',np.linspace(0,0.5,50)),
        ]:
        showPlotComparisonsFor(df, var,varLabel,bins, f'{outdir}/Run{r}')

In [ ]:
def compareCalib(var,vartitle,outdir,runs=[1726304466,1726339135,1726512750]):

    modules = data[runs[0]]['Typecode'].unique()
    for m in modules:

        fig, ax = plt.subplots(figsize=(12,6))
        for r in runs:
            mask=(data[r]['Typecode'].str.find(m)>=0)
            ax.plot(data[r][mask]['Channel'],data[r][mask][var],label=f'{r}')
        ax.grid()
        ax.text(0.05,0.1,m,transform=ax.transAxes)
        ax.set_xlabel('Channel')
        ax.set_ylabel(vartitle)
        ax.legend(loc='upper left')
        fig.tight_layout()
        fig.savefig(f'{outdir}/{m}_{var}.png')
        plt.close()
    
for var,vartitle in [
        ('ADC_ped','Pedestal'),
        ('cm2_slope','rADC vs $CM_{2}$ slope'),
        ('ADC_rms','RMS(ADC)'),
        ('adcm1_slope',r'ADC vs $CM_{all}$ slope')
    ]:
    compareCalib(var,vartitle,outdir)
    

In [ ]:
def _noiseModel(x,a,b,c): #,k):
    """assumes that the noise scales quadratically with the capacitance (or the cell area)"""
    cap=x
    #cap=x[:,0]
    #trace=x[:,1]
    return (a*(cap**2)+b*cap+c) #*(1+k*trace)

def runNoiseModelFit(df,var,outfile):

    mask = (df['Valid']==1) & (df['AreaSF']>0)
    x=df[mask]['AreaSF'].values.astype(float)
    xs=np.linspace(0.1,1.3,100)
    y=df[mask][var].values.astype(float)
    popt, pcov = curve_fit(_noiseModel, x, y, bounds=([0,0,0], [2., 2., 5.]))
    yvars=[]
    for i in range(len(popt)):
        popt_up=popt.copy()
        popt_up[i]=popt_up[i]+np.sqrt(pcov[i][i])
        yvars.append(_noiseModel(xs,*popt_up))

        popt_dn=popt.copy()
        popt_dn[i]=popt_dn[i]-np.sqrt(pcov[i][i])
        yvars.append(_noiseModel(xs,*popt_dn))
    yvars=np.array(yvars)

    #show plot
    fig, ax = plt.subplots(figsize=(8,8))

    #expected
    adc2fC=0.238 #typical adc2fC
    Cap=xs*45.  #capacitance of 300 um
    exp_param=(0.000017*(Cap**2)+0.002119*Cap+0.190295)/adc2fC
    ax.plot(xs,exp_param, lw=2, ls='--', c='gray',label='expected')

    #observed
    H, xedges, yedges = np.histogram2d(x, y, bins=(np.linspace(0.1,1.5,40), np.linspace(0,5,50)))
    hep.hist2dplot(H, xedges, yedges, labels=False,cmin=1)
    ax.fill_between(xs,yvars.min(axis=0), yvars.max(axis=0),alpha=0.2)
    ax.plot(xs,_noiseModel(xs,*popt),lw=2,ls='-',c='r',label='fit')

    txt_kwargs={'transform':ax.transAxes, 'fontsize':14}
    ax.text(0.05,0.8,f'#channels used = {mask.sum()}', **txt_kwargs) 
    ax.text(0.05,0.75, fr'$Noise~={popt[0]:3.3f}\cdot SF^2+{popt[1]:3.3f}\cdot SF+{popt[2]:3.3f}$',  **txt_kwargs) 

    ax.legend(loc='upper left')
    ax.grid()
    ax.set_ylabel('Noise')
    #ax.set_ylim(0.5,5)
    ax.set_xlabel('Cell area / Full cell area')
    fig.tight_layout()
    plt.savefig(outfile)
    plt.close()
    
for r in [1726304466,1726339135,1726512750]:
    for var in ['ADC_rms','En_rms']:
        runNoiseModelFit(data[r],var,f'{outdir}/Run{r}_{var}_noisescaling')